<a href="https://colab.research.google.com/github/n-bzy/iannwtf/blob/main/incomplete_homework07_Silvie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**INPUT to CNN**:  <br>
4-tuple/quadruple containing 4 mnist images of size mxn: (digit1,digit2,digit3,digit4) <br><br>
**OUTPUT of CNN & INPUT to RNN/LSTM**: <br>
4-tuple containing 4 one-hot vectors representing the digit depicted on each of the 4 images <br><br>
**TARGET & OUTPUT of RNN/LSTM**: <br>
4-tuple containing 4 float/int values corresponding to the cummulative sum results

--------------------------------------------------------------------------------
**RNN**: many-to-many model

# Import data set

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

#load MNIST data
train_ds, val_ds = tfds.load('mnist', split=['train', 'test'], as_supervised=True)

# Preprocessing step

In [ ]:
def preprocessing(data):
    """Preparing data sets for use in network"""

    #change datatype to float
    data = data.map(lambda x,t : (tf.cast(x, tf.float32), t))
    #normalize the image pixel values
    data = data.map(lambda x,t : ((x/128.)-1, t))

    #get 4 datasets with data samples in random order
    digit_1 = data.shuffle(2000)
    digit_2 = data.shuffle(2000)
    digit_3 = data.shuffle(2000)
    digit_4 = data.shuffle(2000)

    #create sequences of 4 digits each that will be fed as INPUT into the model
    #zip digit 1-4: map (x1,t1), (x2,t2), (x3,t3), (x4,t4) to ((x1,t1),(x2,t2),(x3,t3),(x4,t4))
    zipped_ds = tf.data.Dataset.zip((digit_1,digit_2,digit_3,digit_4))

    #prepare target:create 5-tuple (x1,x2,x3,x4,t) with t = array containing the 4 original targets
    zipped_ds = zipped_ds.map(lambda digit_1,digit_2,digit_3,digit_4: (digit_1[0], digit_2[0], digit_3[0], digit_4[0], [digit_1[1],digit_2[1],digit_3[1],digit_4[1]]))

    #adjust input to single 4-tuple: (input,target) with input = (img1,img2,img3,img4)
    zipped_ds = zipped_ds.map(lambda digit_1,digit_2,digit_3,digit_4, t: ((digit_1, digit_2, digit_3, digit_4), t))
    
    #adjust target to an array with the 4 cumulative sums
    #create array of indices
    indices = tf.range(4)
    #make every 2nd series target (-> has odd index) negative
    zipped_ds = zipped_ds.map(lambda input,t: (input, tf.where(tf.math.floormod(indices,2)==0, t, -t)))
    #compute the cumulative sums
    zipped_ds = zipped_ds.map(lambda input,t: (input, tf.math.cumsum(t)))

    #shuffle(?neccessary), batch, prefetch data
    preprocessed_ds = zipped_ds.shuffle(1000).batch(256).prefetch(tf.data.AUTOTUNE)

    return preprocessed_ds

#-------------------------------------------------------
# Test preprocessing and check for value and target shapes 
# (important for initializing of model with input values)
train_test = preprocessing(train_ds)
for input,t in train_test.take(1):
    print("Input: " + str(tf.shape(input)) + " -> [#seriesInputElements, batchSize, xImgDimension, yImgDimension, usingGrayValues/noColorChannels]")
    print("Target: " + str(tf.shape(t)) + " -> [batchSize, #seriesOutputElements]")

Input: tf.Tensor([  4 256  28  28   1], shape=(5,), dtype=int32) -> [#seriesInputElements, batchSize, xImgDimension, yImgDimension, usingGrayValues/noColorChannels]
Target: tf.Tensor([256   4], shape=(2,), dtype=int32) -> [batchSize, #seriesOutputElements]


# CNN model

In [ ]:
class CNN(tf.keras.Model):
  """Basic CNN structure"""
  def __init__(self):
    """Initialize CNN blocks, pooling, optimizer, loss function"""
    super(CNN, self).__init__()

    #CNN block 1
    self.convlayer1 = tf.keras.layers.Conv2D(filters=24, kernel_size=3, padding='same', activation='relu', input_shape=input_shape[2:])
    self.convlayer2 = tf.keras.layers.Conv2D(filters=24, kernel_size=3, padding='same', activation='relu')
    #pooling layer: reduce img size to 16x16
    self.pool1 = tf.keras.layers.TimeDistributed(tf.keras.layers.MaxPooling2D(pool_size=2, strides=2))

    #CNN block 2
    self.convlayer3 = tf.keras.layers.Conv2D(filters=48, kernel_size=3, padding='same', activation='relu')
    self.convlayer4 = tf.keras.layers.Conv2D(filters=48, kernel_size=3, padding='same', activation='relu')
    #global pooling layer: reduce img size to 1x1 for dense layer
    self.global_pool = tf.keras.layers.TimeDistributed(tf.keras.layers.GlobalAvgPool2D())

    #output layer
    self.out = tf.keras.layers.Dense(10, activation='softmax') #we want a one-hot vector as output

  def call(self, x):
    """Forward propagation step for CNN"""
    x = self.convlayer1(x)
    x = self.convlayer2(x)
    x = self.pool1(x)
    x = self.convlayer3(x)
    x = self.convlayer4(x)
    x = self.global_pool(x)
    y = self.out(x)
    return y


# LSTM model

In [ ]:
class LSTMCell(tf.keras.layers.AbstractRNNCell):
  """Create a LSTM cell"""
  def __init__(self, hidden_state, cell_state):
    super().__init__()

    self.

# RNN model

# Training step

In [ ]:
#create CNN model
cnn_model = CNN()
#create RNN model
rnn_model = RNN()

# Visualization